## CAŁKOWANIE NUMERYCZNE

In [1]:
import numpy as np
from typing import Callable, Tuple, List

### 1) metoda trapezów
$$I = \sum_{i=0}^{n-1} I_i = \dfrac{h}{2} \sum_{i=0}^{n-1} [f(x_i)+f(x_{i+1})]$$

gdzie: $\quad h = \dfrac{b-a}{n}$

In [2]:
def trapez_integration(func: Callable[[float], float], a: float, b: float, n: int) -> float:
    h = (b - a) / n 
    x = np.linspace(a, b, n+1)
    y = func(x)        
    result = np.sum(y[:-1] + y[1:]) * h/2
    return result

#### 2) metoda Simpsona
$$S(f) = \sum_{i=0}^{\frac{n}{2}-1} S_{2i}(f) = \dfrac{h}{3} \sum_{i=0}^{\frac{n}{2}-1} [f_{2i}+4f_{2i+1}+f_{2i+2}] =  \dfrac{h}{3} \sum_{i=0}^{\frac{n}{2}-1} [f_{0}+ f_n+ 2(f_2+f_4+\ldots+f_{n-2})+4(f_1+f_3+\ldots+f_{n-1})]$$

gdzie: $\quad h = \dfrac{b-a}{n}$, $\quad n - \text{liczba węzłów,} \textbf{parzyste !}$

Warto pamiętać, że dokładność wzoru Simpsona zależy od tego, jak dobrze funkcja pasuje do parabolicznej aproksymacji. 

In [3]:
def simp_intergration(func: Callable[[float], float], a: float, b: float, n: int) -> float:
    h = (b-a)/n
    x = np.linspace(a, b, n+1)
    y = func(x)
    if n%2 != 0:
        raise ValueError('n musi być parzyste !')
    result = h/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])
    return result

### 3) metoda Gaussa - Legendre'a:

#### a) wielomian Legendre'a:
$ P_{n+1}(x) = \dfrac{2n+1}{n+1} x P_n(x) - \dfrac{n}{n+1} P_{n-1}(x), \quad \text{gdzie} \quad P_0 = 1, \quad P_1 = x$.

$ P'_{n+1}(x) = \dfrac{n}{x^2-1} \left(x P_n(x) - P_{n-1}(x)\right)$

#### b) kwadratura Gaussa-Legendre'a:
$$ A_i = \dfrac{2}{(1-x_i^2)[P^\prime_{n+1}(x_i)]^2} $$


#### c) zmiana zakresu $(a,b)$, na zakres "standardowy" $(-1,1)$:
$$x = \dfrac{b+a}{2} + \dfrac{b-a}{2} \xi \quad \Rightarrow \quad dx = \dfrac{b-a}{2} d\xi$$

#### d) ostatecznie wzór:
$$ \int_a^b f(x) dx = \dfrac{b-a}{2} \sum _{i=1}^n A_i f(x_i) $$

In [4]:
def wiel_legendre(x: float, n: int) -> Tuple[float, float]:
    p = np.zeros(n + 1)
    p[0], p[1] = 1, x 
    for i in range(2, n + 1):
        p[i] = ((2 * i - 1) * x * p[i - 1] - (i - 1) * p[i - 2]) / i
    dp = n / (x**2-1) * (x * p[n] - p[n - 1])
    return p[n], dp

In [5]:
def gauss_legendre_integral(func: Callable[[float], float], a: float, b: float, n: int) -> float:
    ksi = np.zeros(n)
    Ai = np.zeros(n)

    for i in range(n):
        x0 = np.cos(np.pi * (i + 0.75) / (n + 0.5)) 
        for _ in range(10):
            p, dp = wiel_legendre(x0, n)
            x0 -= p / dp
        ksi[i] = x0
        Ai[i] = 2 / ((1 - x0**2) * dp**2)

    x = (b + a) / 2 + (b - a) / 2 * ksi
    integral = (b-a)/2 * np.sum(Ai * func(x))
    return integral

## RÓŻNICZKOWANIE

### 1) ekstrapolacja Richardsona
- $D_{f1}: \quad$ $D_{f1} = \dfrac{f(x+h)-f(x)}{h}$

- $D_{c2}: \quad$
$D_{c2} = \dfrac{f(x+h)-f(x-h)}{2h}$

- $D_{c4}: \quad$ $D_{c4} = \dfrac{2^2 D_{c2}(x,h) - D_{c2}(x,2h)}{3} = \dfrac{8f(x+h)-8f(x-h)-f(x+2h)+f(x-2h)}{12h}$

In [ ]:
def Df1(func: Callable[[float], float], x: float, h: float) -> float:
    return (func(x + h) - func(x)) / (h)

In [ ]:
def Dc2(func: Callable[[float], float], x: float, h: float) -> float:
    return (func(x + h) - func(x - h)) / (2 * h)

In [ ]:
def Dc4(func: Callable[[float], float], x: float, h: float) -> float:
    return (-func(x + 2*h) +8*func(x+h) - 8*func(x - h)+ func(x-2*h)) / (12 * h)

### 2) wzór trójpunktowy:
$$f'(x) = \dfrac{f(x+h)-f(x-h)}{2h}$$

$$f''(x) = \dfrac{f(x+h)-2f(x)+f(x-h)}{h^2}$$